<a href="https://colab.research.google.com/github/JPBC27/Apps-Script-Prueba/blob/main/Proceso_Eval_Desempe%C3%B1o_SPSA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
import os
import pandas as pd
import numpy as np
import shutil
from datetime import datetime
import sys

def split_competencies(competencies_string):
  if pd.isna(competencies_string):
    return [], [], [], []
  competencies = competencies_string.split(' -')
  #print(competencies)
  if len(competencies) == 4:
      return competencies[0].strip(), competencies[1].strip(), competencies[2].strip(), competencies[3].strip()
  elif len(competencies) == 3:
      return competencies[0].strip(), competencies[1].strip(), competencies[2].strip(), ''
  elif len(competencies) == 2:
      return competencies[0].strip(), competencies[1].strip(), '',''
  elif len(competencies) == 1:
    return competencies[0].strip(), '', '',''
  else:
      return '', '', '',''

def Buscar_Archivos(folder_path):
  drive.mount('/content/drive')
  file_estructura=''
  file_evalMoni=''
  file_closurvey=''
  file_estrdig=''
  file_gorgos=''

  if os.path.exists(folder_path):
    files = os.listdir(folder_path)
    print("Archivos en la carpeta:")
    for file in files:
      if file.startswith('6.- Estructura de personal'):
        file_estructura = file
        print(file_estructura)
      elif file.startswith('evaluation_monitoring'):
        file_evalMoni=file
        print(file_evalMoni)
      elif file.startswith('closing_survey_monitoring'):
        file_closurvey=file
        print(file_closurvey)
      elif file.startswith('7.- Estructura de personal'):
        file_estrdig=file
        print(file_estrdig)
      elif file.startswith('Base GOR'):
        file_gorgos = file
        print(file_gorgos)
  else:
    print(f"La carpeta '{folder_path}' no existe.")
  return file_estructura, file_evalMoni, file_closurvey, file_estrdig, file_gorgos,[file_estructura, file_evalMoni, file_closurvey, file_estrdig, file_gorgos]
#Busqueda de archivos

def RealizarBackup(folder_path,folder_path_hist,files_to_backup):
  today = datetime.today().strftime('%Y-%m-%d')
  backup_folder = os.path.join(folder_path_hist, today)

  if not os.path.exists(backup_folder):
      os.makedirs(backup_folder)
      print(f"Carpeta '{backup_folder}' creada exitosamente.")
  else:
      print(f"La carpeta '{backup_folder}' ya existe.")

  for file in files_to_backup:
    source_file = os.path.join(folder_path, file)
    destination_file = os.path.join(backup_folder, file)

    if os.path.exists(source_file):
      try:
        shutil.copy2(source_file, destination_file)
        print(f"Archivo '{file}' copiado a '{backup_folder}'")
      except Exception as e:
        print(f"Error al copiar '{file}': {e}")
    else:
      print(f"Archivo '{file}' no encontrado en la carpeta de origen.")


def CargarEvalMonitoring(file_evalMoni,folder_path):

  file_name = file_evalMoni
  file_path = os.path.join(folder_path, file_name)

  if os.path.exists(file_path):
    try:
      df = pd.read_excel(file_path, sheet_name='Listado de Evaluaciones', usecols='A:T', skiprows=7, dtype=str)
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
  else:
    print(f"El archivo '{file_name}' no existe en la carpeta especificada.")

  df = df.rename(columns={'Identifier.1': 'Identifier_Evaluador','Nombre.1': 'Nombre_Evaluador','Identifier.1': 'Identifier_Evaluador','Apellido.1': 'Apellido_Evaluador','Rut.1': 'Rut_Evaluador','Username.1': 'Username_Evaluador',
                          'Email.1': 'Email_Evaluador','Área.1': 'Área_Evaluador','Cargo.1': 'Cargo_Evaluador'})
  df = df.loc[df['Categoría']=='Evaluación del Jefe directo']
  df_base = df[['Identifier', 'Nombre', 'Apellido', 'Rut','Username', 'Email', 'Área', 'Cargo', 'Jerarquía', 'Familia de competencias', 'Identifier_Evaluador', 'Nombre_Evaluador',
        'Apellido_Evaluador', 'Rut_Evaluador', 'Username_Evaluador', 'Email_Evaluador', 'Área_Evaluador', 'Cargo_Evaluador']]
  df_base.drop_duplicates(inplace=True)
  df_base[['Area P1', 'Area P2', 'Area P3','Area P4']] = df_base['Área'].apply(lambda x: pd.Series(split_competencies(x)))
  df_base['Area P1'] = df_base['Area P1'].str.upper()
  df_clean = df_base.copy()
  df_clean = df_clean[~df_clean['Familia de competencias'].isin(['decs-pva-31287','decs-admi-31287'])]
  df_clean = df_clean[df_clean['Area P1'].isin(['MK','FRET','OPL','SPO','DF'])]
  print("Evaluation Monitoring Procesado ....")
  return df_clean

def CargaEstructuraTiendas(file_estructura,folder_path):
  file_name = file_estructura
  file_path = os.path.join(folder_path, file_name)
  if os.path.exists(file_path):
    try:
      df_estructura = pd.read_excel(file_path, sheet_name='Sheet1', usecols='A:AG', skiprows=2,dtype=str)
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
  else:
    print(f"El archivo '{file_name}' no existe en la carpeta especificada.")

  df_estructura_base = df_estructura[['Número de persona','Nombre de unidad de negocio','Número de documento de identidad principal','Nombre.1','Fecha de inicio de relación laboral']]
  df_estructura_base['Fecha de inicio de relación laboral'] = pd.to_datetime(df_estructura_base['Fecha de inicio de relación laboral'], errors='coerce').dt.strftime('%d/%m/%Y')
  df_estructura_base.rename(columns={'Nombre.1': 'Nombre'},inplace=True)
  print("Estructura Tiendas Procesado ....")
  return df_estructura_base

def CargaEstructuraDig(file_estrdig,folder_path):
  file_name = file_estrdig
  file_path = os.path.join(folder_path, file_name)
  if os.path.exists(file_path):
    try:
      df_estructura_dig = pd.read_excel(file_path, sheet_name='Sheet1', usecols='A:AG', skiprows=2,dtype=str)
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
  else:
    print(f"El archivo '{file_name}' no existe en la carpeta especificada.")

  df_estructura_base_dig = df_estructura_dig[['Número de persona','Nombre de unidad de negocio','Número de documento de identidad principal','Nombre','Fecha de inicio de relación laboral']]
  df_estructura_base_dig['Fecha de inicio de relación laboral'] = pd.to_datetime(df_estructura_dig['Fecha de inicio de relación laboral'], errors='coerce').dt.strftime('%d/%m/%Y')
  print("Estructura Digital Procesado ....")
  return df_estructura_base_dig

def ConsolEstructura(df_estructura_base,df_estructura_base_dig):
  df_estructura_consol = pd.concat([df_estructura_base, df_estructura_base_dig], ignore_index=True)
  df_estructura_consol = df_estructura_consol.loc[~df_estructura_consol['Número de persona'].isna()]
  df_estructura_consol['Nombre'] = df_estructura_consol['Nombre'].str.title()
  df_estructura_consol.rename(columns={'Nombre':'Nombre Completo' },inplace=True)
  return df_estructura_consol

def CargaGorgos(file_gorgos,folder_path):
  file_name = file_gorgos
  file_path = os.path.join(folder_path, file_name)

  if os.path.exists(file_path):
    try:
      df_gorgos = pd.read_excel(file_path, sheet_name='Tiendas', usecols='A:F', skiprows=0,dtype=str)
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
  else:
    print(f"El archivo '{file_name}' no existe en la carpeta especificada.")

  df_gorgos_base = df_gorgos[['Tiendas','GOR','GOS']]
  df_gorgos_base['Tiendas']=df_gorgos_base['Tiendas'].str.strip()
  df_gorgos_base['Tienda M']=df_gorgos_base['Tiendas'].str.upper()
  df_gorgos_base.drop_duplicates(inplace=True)
  return df_gorgos_base

def CargaRespuestaEvalMoni(file_evalMoni,folder_path):
  file_name = file_evalMoni
  file_path = os.path.join(folder_path, file_name)
  import pandas as pd
  if os.path.exists(file_path):
    try:
      df_notas_1_4 = pd.read_excel(file_path, sheet_name='Evaluados', usecols='A:AS', skiprows=7, dtype=str)
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
  else:
    print(f"El archivo '{file_name}' no existe en la carpeta especificada.")

  df_notas_1_4.rename(columns={'Asignadas':'Asignadas EJD','Completadas':'Completadas EJD','En progreso':'En progreso EJD','No comenzado':'No comenzado EJD','% de avance':'% de avance EJD','Ponderación':'Ponderación EJD',
                      'Asignadas.1':'Asignadas EAL','Completadas.1':'Completadas EAL','En progreso.1':'En progreso EAL','No comenzado.1':'No comenzado EAL','% de avance.1':'% de avance EAL','Ponderación.1':'Ponderación EAL',
                      'Asignadas.2':'Asignadas PCI','Completadas.2':'Completadas PCI','En progreso.2':'En progreso PCI','No comenzado.2':'No comenzado PCI','% de avance.2':'% de avance PCI','Ponderación.2':'Ponderación PCI',
                      'Asignadas.3':'Asignadas EME','Completadas.3':'Completadas EME','En progreso.3':'En progreso EME','No comenzado.3':'No comenzado EME','% de avance.3':'% de avance EME','Ponderación.3':'Ponderación EME',
                      'Asignadas.4':'Asignadas EMC','Completadas.4':'Completadas EMC','En progreso.4':'En progreso EMC','No comenzado.4':'No comenzado EMC','% de avance.4':'% de avance EMC','Ponderación.4':'Ponderación EMC',
                      'Estado':'Estado AUT','Ponderación.5':'Ponderación AUT',
                      'Asignadas.5':'Asignadas FED','Completadas.5':'Completadas FED','% de avance.5':'% de avance FED'
                      },inplace=True)
  return df_notas_1_4

def CargaClosingSurvey(file_closurvey,folder_path):
  file_name = file_closurvey
  file_path = os.path.join(folder_path, file_name)
  import pandas as pd
  if os.path.exists(file_path):
    try:
      df_closurv = pd.read_excel(file_path, sheet_name='Feedbacks', usecols='A:I', skiprows=7, dtype=str)

    except Exception as e:
        print(f"Error al leer el archivo: {e}")
  else:
    print(f"El archivo '{file_name}' no existe en la carpeta especificada.")
  return df_closurv

def DataProcessing(df_evalMonitoring,df_estructura_consol,df_gorgos_base,df_notas_1_4,df_closurv):
  df_clean_2 = pd.merge(df_evalMonitoring, df_estructura_consol[['Número de persona','Nombre Completo','Fecha de inicio de relación laboral']], left_on='Identifier', right_on='Número de persona', how='left')
  df_clean_2.drop(columns=['Número de persona'], inplace=True)

  df_estructura_base_2 = df_estructura_consol.copy()
  df_estructura_base_2.rename(columns={'Fecha de inicio de relación laboral':'Fecha de inicio de relación laboral Evaluador' },inplace=True)
  df_clean_2 = pd.merge(df_clean_2, df_estructura_base_2[['Número de persona', 'Fecha de inicio de relación laboral Evaluador']], left_on='Identifier_Evaluador', right_on='Número de persona', how='left')
  df_clean_2.drop(columns=['Número de persona'], inplace=True)
  df_clean_2.loc[df_clean_2['Area P4']!='','Area P3'] = df_clean_2.loc[df_clean_2['Area P4']!='','Area P3'] +' '+df_clean_2.loc[df_clean_2['Area P4']!='','Area P4']
  df_clean_2.rename(columns={'Area P1':'Formato','Area P2':'Tienda','Area P3':'Seccion' },inplace=True)
  df_clean_2['Evaluado Activo Cesado'] = np.where(df_clean_2['Fecha de inicio de relación laboral'].isnull(), 'Cesado', 'Activo')
  df_clean_2['Evaluador Activo Cesado'] = np.where(df_clean_2['Fecha de inicio de relación laboral Evaluador'].isnull(), 'Cesado', 'Activo')
  df_clean_2.drop_duplicates(inplace=True)

  df_clean_3 = pd.merge(df_clean_2, df_notas_1_4[['Identifier','% de avance EJD','Estado AUT','% de avance EME','% de avance FED']], left_on='Identifier', right_on='Identifier', how='left')
  df_clean_4 = pd.merge(df_clean_3, df_closurv[['Identificador','Estado encuesta']], left_on='Identifier', right_on='Identificador', how='left')
  df_clean_4.drop(columns=['Identificador'], inplace=True)

  #Autoevaluación
  df_clean_4['Autoevaluación Status'] = 'N/A'
  df_clean_4['Autoevaluación %'] = '0%'
  df_clean_4['Autoevaluación Den'] = 0
  df_clean_4['Autoevaluación Num'] = 0
  df_clean_4.loc[df_clean_4['Estado AUT']==' - ','Autoevaluación Status'] = 'NO APLICA'
  df_clean_4.loc[df_clean_4['Autoevaluación Status']=='N/A','Autoevaluación Status'] = df_clean_4.loc[df_clean_4['Autoevaluación Status']=='N/A','Estado AUT']
  df_clean_4.loc[df_clean_4['Autoevaluación Status']=='NO APLICA','Autoevaluación %'] = 'NO APLICA'
  df_clean_4.loc[df_clean_4['Autoevaluación Status']!='NO APLICA','Autoevaluación Den'] = 1
  df_clean_4.loc[df_clean_4['Autoevaluación Status']=='Finalizado','Autoevaluación %'] = '100%'
  df_clean_4.loc[df_clean_4['Autoevaluación Status']=='Finalizado','Autoevaluación Num'] = 1
  ## Evaluacion Lider
  df_clean_4['Evaluación del Líder Status'] = 'No comenzado'
  df_clean_4['Evaluación del Líder Den'] = 1
  df_clean_4['Evaluación del Líder Num'] = 0
  df_clean_4.loc[df_clean_4['% de avance EJD']=='100%','Evaluación del Líder Status'] = 'Finalizado'
  df_clean_4['Evaluación del Líder %'] = df_clean_4['% de avance EJD']
  df_clean_4.loc[df_clean_4['Evaluación del Líder Status']=='Finalizado','Evaluación del Líder Num'] = 1
  #Matricial
  df_clean_4['Matricial Status'] = 'No comenzado'
  df_clean_4['Matricial %'] = '0%'
  df_clean_4['Matricial Den'] = 0
  df_clean_4['Matricial Num'] = 0
  df_clean_4.loc[df_clean_4['% de avance EME']==' - ','Matricial Status'] = 'NO APLICA'
  df_clean_4.loc[df_clean_4['% de avance EME']=='100%','Matricial Status'] = 'Finalizado'
  df_clean_4.loc[df_clean_4['Matricial Status']=='NO APLICA','Matricial %'] = 'NO APLICA'
  df_clean_4.loc[df_clean_4['Matricial Status']!='NO APLICA','Matricial Den'] = 1
  df_clean_4.loc[df_clean_4['Matricial Status']=='Finalizado','Matricial %'] = '100%'
  df_clean_4.loc[df_clean_4['Matricial Status']=='Finalizado','Matricial Num'] = 1
  ## Feedback
  df_clean_4['Feedback Status'] = 'No comenzado'
  df_clean_4['Feedback Den'] = 1
  df_clean_4['Feedback Num'] = 0
  df_clean_4.loc[df_clean_4['% de avance FED']=='100%','Feedback Status'] = 'Finalizado'
  df_clean_4['Feedback %'] = df_clean_4['% de avance FED']
  df_clean_4.loc[df_clean_4['Feedback Status']=='Finalizado','Feedback Num'] = 1
  ## Encuesta
  df_clean_4['Encuesta %'] ='0%'
  df_clean_4['Encuesta Status'] = df_clean_4['Estado encuesta']
  df_clean_4.loc[df_clean_4['Estado encuesta']=='Finalizado','Encuesta %'] = '100%'
  df_clean_4['Encuesta Den'] = 1
  df_clean_4['Encuesta Num'] = 0
  df_clean_4.loc[df_clean_4['Estado encuesta']=='Finalizado','Encuesta Num'] = 1

  df_clean_4['Den Avance Global'] = df_clean_4['Autoevaluación Den']+df_clean_4['Evaluación del Líder Den']+df_clean_4['Matricial Den']+df_clean_4['Feedback Den']+df_clean_4['Encuesta Den']
  df_clean_4['Num Avance Global'] = df_clean_4['Autoevaluación Num']+df_clean_4['Evaluación del Líder Num']+df_clean_4['Matricial Num']+df_clean_4['Feedback Num']+df_clean_4['Encuesta Num']
  df_clean_4['%Avance Global'] = np.where(df_clean_4['Num Avance Global']/df_clean_4['Den Avance Global']==1,1,0)#*100
  df_clean_4['Status Avance Global'] = np.where(df_clean_4['%Avance Global']==1,'Finalizado','Pendiente')#*100

  df_clean_4['Tienda M']=df_clean_4['Tienda'].str.upper()
  df_clean_4['Tienda M']=df_clean_4['Tienda M'].str.strip()

  df_clean_5 = pd.merge(df_clean_4, df_gorgos_base, left_on='Tienda M', right_on='Tienda M', how='left')
  list_cabecera_fin =['Identifier', 'Nombre', 'Apellido', 'Rut', 'Username','Nombre Completo','Email', 'Área',
        'Cargo', 'Jerarquía', 'Familia de competencias', 'Identifier_Evaluador',
        'Nombre_Evaluador', 'Apellido_Evaluador', 'Rut_Evaluador',
        'Username_Evaluador', 'Email_Evaluador', 'Área_Evaluador',
        'Cargo_Evaluador', 'Formato', 'Tienda', 'Seccion','Fecha de inicio de relación laboral',
        'Fecha de inicio de relación laboral Evaluador', 'Evaluado Activo Cesado',
        'Evaluador Activo Cesado','GOR','GOS', 'Autoevaluación Status','Evaluación del Líder Status','Matricial Status','Feedback Status','Encuesta Status',
        'Autoevaluación %','Evaluación del Líder %','Matricial %','Feedback %','Encuesta %',
        'Den Avance Global','Num Avance Global','%Avance Global','Status Avance Global']
  df_clean_final = df_clean_5[list_cabecera_fin]
  return df_clean_final

def GuardarExcel(df_clean_final,folder_ouput):
  today = datetime.today().strftime('%Y-%m-%d')
  output_file_path = f'{folder_ouput}/output_{today}.xlsx'
  df_clean_final.to_excel(output_file_path, index=False, engine='openpyxl')
  print(f"DataFrame exported to: {output_file_path}")

if __name__ == "__main__":

  folder_path = '/content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Inputs'
  folder_path_hist = '/content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Inputs/Inputs Historico'
  folder_ouput = '/content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Outputs'

  file_estructura, file_evalMoni, file_closurvey, file_estrdig, file_gorgos, list_bkp = Buscar_Archivos(folder_path)
  RealizarBackup(folder_path,folder_path_hist,list_bkp)
  df_evalMonitoring = CargarEvalMonitoring(file_evalMoni,folder_path)
  df_estructuraTienda = CargaEstructuraTiendas(file_estructura,folder_path)
  df_estructura_dig = CargaEstructuraDig(file_estrdig,folder_path)
  df_estructura_consol = ConsolEstructura(df_estructuraTienda,df_estructura_dig)
  df_gorgos_base = CargaGorgos(file_gorgos,folder_path)
  df_notas_1_4 = CargaRespuestaEvalMoni(file_evalMoni,folder_path)
  df_closurv = CargaClosingSurvey(file_closurvey,folder_path)
  df_final = DataProcessing(df_evalMonitoring,df_estructura_consol,df_gorgos_base,df_notas_1_4,df_closurv)
  GuardarExcel(df_final,folder_ouput)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos en la carpeta:
Base GOR GOS.xlsx
6.- Estructura de personal - Colaboradores FRET + FREG + PVO + Makro + CHD + Oslo 31.01.xlsx
closing_survey_monitoring__2025-01-31_10_12_15_-0300.xlsx
evaluation_monitoring__2025-01-31_10_12_06_-0300.xlsx
7.- Estructura de personal - Digital Foods 29.01.xlsx
La carpeta '/content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Inputs/Inputs Historico/2025-01-31' ya existe.
Archivo '6.- Estructura de personal - Colaboradores FRET + FREG + PVO + Makro + CHD + Oslo 31.01.xlsx' copiado a '/content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Inputs/Inputs Historico/2025-01-31'
Archivo 'evaluation_monitoring__2025-01-31_10_12_06_-0300.xlsx' copiado a '/content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Inputs/Inputs Historico/2025-01-31'
Archivo 'closing_survey_monitoring__2025-01-31_10_12_15_-0300.xlsx' copiado a '/content/drive/MyDriv

<ipython-input-3-bfa305ef1fdd>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base.drop_duplicates(inplace=True)
<ipython-input-3-bfa305ef1fdd>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base[['Area P1', 'Area P2', 'Area P3','Area P4']] = df_base['Área'].apply(lambda x: pd.Series(split_competencies(x)))
<ipython-input-3-bfa305ef1fdd>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

Evaluation Monitoring Procesado ....


<ipython-input-3-bfa305ef1fdd>:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estructura_base['Fecha de inicio de relación laboral'] = pd.to_datetime(df_estructura_base['Fecha de inicio de relación laboral'], errors='coerce').dt.strftime('%d/%m/%Y')
<ipython-input-3-bfa305ef1fdd>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estructura_base.rename(columns={'Nombre.1': 'Nombre'},inplace=True)
<ipython-input-3-bfa305ef1fdd>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Estructura Tiendas Procesado ....
Estructura Digital Procesado ....


<ipython-input-3-bfa305ef1fdd>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gorgos_base['Tiendas']=df_gorgos_base['Tiendas'].str.strip()
<ipython-input-3-bfa305ef1fdd>:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gorgos_base['Tienda M']=df_gorgos_base['Tiendas'].str.upper()
<ipython-input-3-bfa305ef1fdd>:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

DataFrame exported to: /content/drive/MyDrive/SPSA/Prueba Eval Desempeño/Outputs/output_2025-01-31.xlsx
